In [6]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
class MyLSTM(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs
        
    def build(self, input_shape):
        self.kernel = self.add_variable("kernel",
                                        shape=[int(input_shape[-1]),
                                               self.num_outputs])

    def call(self, input):
        return tf.matmul(input, self.kernel)

layer = MyDenseLayer(10)
a= layer(tf.zeros([10, 5]))


10


In [9]:
class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self, kernel_size, filters):
        super(ResnetIdentityBlock, self).__init__(name='')
        filters1, filters2, filters3 = filters

        self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
        self.bn2a = tf.keras.layers.BatchNormalization()

        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()

        self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
        self.bn2c = tf.keras.layers.BatchNormalization()

        def call(self, input_tensor, training=False):
            x = self.conv2a(input_tensor)
            x = self.bn2a(x, training=training)
            x = tf.nn.relu(x)

            x = self.conv2b(x)
            x = self.bn2b(x, training=training)
            x = tf.nn.relu(x)

            x = self.conv2c(x)
            x = self.bn2c(x, training=training)

            x += input_tensor
            return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
class MyLSTM(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs
        
    def build(self, input_shape):
        self.kernel = self.add_variable("kernel",
                                        shape=[int(input_shape[-1]),
                                               self.num_outputs])

    def call(self, input):
        return tf.matmul(input, self.kernel)

layer = MyDenseLayer(10)
a= layer(tf.zeros([10, 5]))

In [1]:
## try to use easydict!!!!!!!!!!!
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import os,sys,time
import torch
import options,util
from config import get_config
from graphs.get_graph import get_graph
from systems.get_system import get_system
from ipdb import set_trace as debug
import tensorflow as tf
import tensorflow as tf
import numpy as np

LSTM_Cell=tf.keras.layers.LSTMCell(units=8, activation='tanh', \
                              recurrent_activation='hard_sigmoid',\
                              use_bias=True, kernel_initializer='glorot_uniform',\
                              recurrent_initializer='orthogonal',\
                              bias_initializer='zeros', unit_forget_bias=True,\
                              kernel_regularizer=None, recurrent_regularizer=None, \
                              bias_regularizer=None, kernel_constraint=None,\
                              recurrent_constraint=None, bias_constraint=None, \
                              dropout=0.0, recurrent_dropout=0.0, implementation=1)
x0 = [0.0, 0.0]
delta_t=0.02
num_time_interval=10
sigma=1.
target_states = np.array([[np.pi, 0.0]])
target_states = np.repeat(target_states,num_time_interval,axis=0)
dynamic = get_system('InvertedPendulum', x0, delta_t, \
    num_time_interval, sigma, target_states,\
    False, False, False, 0.1, 0.1)
# dw_valid = sigma *  \
#         np.random.randn(self._config.valid_size, self._state_dim,\
#                         self._num_time_interval) * self._sqrt_delta_t
dw = sigma *  np.random.randn(32, 2,num_time_interval) * delta_t
dv= np.zeros((32, num_time_interval))
class dynamic_LSTM(tf.keras.Model):
    def __init__(self,output_dim,cell,dynamic,t):
        super(dynamic_LSTM, self).__init__()
        self.cell=cell
        self._dynamic=dynamic
        self.output_dim=output_dim
        self.t=t
        print('output',output_dim)
    def build(self,input_shape):
        assert isinstance(input_shape,list)
        print('input.shape',input_shape[0])
        self.W = self.add_variable("W",shape=[8,1])
        self.b = self.add_variable("b",shape=[1])
        super(dynamic_LSTM, self).build(input_shape)
    def call(self,var):
        assert isinstance(var,list)
        z,y,state0,state1,x,dw,dv=var
        gamma=self._dynamic.compute_gamma(x)
        u=-tf.matmul(tf.reduce_sum(tf.expand_dims(z, -1) * gamma, axis=1, keepdims=False),
                     tf.linalg.inv(self._dynamic.R_tf))
        y = self.dynamic.value_prop(x, y, z, dw[:,:,self.t], u, self.t)
        x = self.dynamic.dynamics_prop(x, u, dw[:,:,self.t], dv[:,self.t])
        states=tf.stack([state0,state1],axis=0)
        z, states=self.cell(x, states)
        z_pred=tf.matmul(z,self.W)+self.b
        var=[z_pred,y,states[0],states[1],x,dw,dv]
        return var
# input.shape [TensorShape([32, 2]), TensorShape([32, 1]),
# TensorShape([2, 32, 8]), TensorShape([32, 2]), TensorShape([32, 2, 10]), TensorShape([32, 10])]
    def compute_output_shape(self, input_shape):
        assert isinstance(input_shape,list)
        shape_a,shape_b,shape_c,shape_d,shape_e,shape_f=input_shape
        return [(2,2), (1, 1),(8, 8),(8,8), (2, 2), ([2, 10],[2,10]),(10, 10)]

all_one_vec=tf.ones(shape=tf.stack([32, 1]))
state0=tf.matmul(all_one_vec,\
                tf.compat.v1.get_variable( name="lstm_init_h_l", \
                shape=[1,8])) 
state1=tf.matmul(all_one_vec,\
        tf.compat.v1.get_variable( name="lstm_init_c_l", \
        shape=[1,8])) 
states=tf.stack([state0,state1],axis=0)
x0=tf.ones((32,2))
t=0
y=np.ones((32,1))
z_init= tf.Variable(tf.random.uniform([1, int(2* 0.5)], minval=-.1, maxval=.1))
z = tf.matmul(all_one_vec, z_init) # outer product - creates a tensor of shape: (batch_size, state_dim/2)
z = tf.concat([tf.zeros_like(z), z], axis=1)
var=[z,y,state0, state1,x0,tf.constant(dw),tf.constant(dv)]

def build(var,dynamic):
    output_shape=[(32, 2), (32, 1),
        (2, 32, 8), (32, 2), (32, 2, 10),(32, 10)]
    LSTM_Cell=tf.keras.layers.LSTMCell(units=8, activation='tanh', \
                              recurrent_activation='hard_sigmoid',\
                              use_bias=True, kernel_initializer='glorot_uniform',\
                              recurrent_initializer='orthogonal',\
                              bias_initializer='zeros', unit_forget_bias=True,\
                              kernel_regularizer=None, recurrent_regularizer=None, \
                              bias_regularizer=None, kernel_constraint=None,\
                              recurrent_constraint=None, bias_constraint=None, \
                              dropout=0.0, recurrent_dropout=0.0, implementation=1)
   
    Mylayer=dynamic_LSTM(output_shape,LSTM_Cell,dynamic,t)
    model=tf.keras.Sequential()
    for i in range(10):
        model.add(Mylayer)
#     for i in range(10):
#         print(i)
#         var=Mylayer(var)
    return model
model=build(var,dynamic)
var=model(var)

W1229 22:47:26.732101 139863610440576 base_layer.py:1814] Layer sequential is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

W1229 22:47:26.733476 139863610440576 deprecation.py:323] From <ipython-input-1-dac6901101b5>:49: Layer.add_variable (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:

output [(32, 2), (32, 1), (2, 32, 8), (32, 2), (32, 2, 10), (32, 10)]
input.shape (32, 2)


AttributeError: 'list' object has no attribute '_keras_mask'

In [87]:

all_one_vec=tf.ones(shape=tf.stack([32, 1]))
state0=tf.matmul(all_one_vec,\
                tf.compat.v1.get_variable( name="lstm_init_h_l", \
                shape=[1,8])) 
state1=tf.matmul(all_one_vec,\
        tf.compat.v1.get_variable( name="lstm_init_c_l", \
        shape=[1,8])) 
states=tf.stack([state0,state1],axis=0)
x0=tf.ones((32,2))
t=0
y=np.ones((32,1))
z_init= tf.Variable(tf.random.uniform([1, int(2* 0.5)], minval=-.1, maxval=.1))
z = tf.matmul(all_one_vec, z_init) # outer product - creates a tensor of shape: (batch_size, state_dim/2)
z = tf.concat([tf.zeros_like(z), z], axis=1)
var=[z,y,states,x0,tf.constant(dw),tf.constant(dv)]
def build(var,dynamic):
    LSTM_Cell=tf.keras.layers.LSTMCell(units=8, activation='tanh', \
                              recurrent_activation='hard_sigmoid',\
                              use_bias=True, kernel_initializer='glorot_uniform',\
                              recurrent_initializer='orthogonal',\
                              bias_initializer='zeros', unit_forget_bias=True,\
                              kernel_regularizer=None, recurrent_regularizer=None, \
                              bias_regularizer=None, kernel_constraint=None,\
                              recurrent_constraint=None, bias_constraint=None, \
                              dropout=0.0, recurrent_dropout=0.0, implementation=1)
   
    Mylayer=dynamic_LSTM(8,LSTM_Cell,dynamic,t)
    model=tf.keras.Sequential()
    for i in range(10):
        model.add(Mylayer)
#     for i in range(10):
#         print(i)
#         var=Mylayer(var)
    return model
model=build(var,dynamic)
var=model(var)

W1229 22:04:42.083417 140222214028160 base_layer.py:1814] Layer sequential_21 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



============= (32, 2)
============= (32, 2)


AttributeError: 'tuple' object has no attribute '_keras_mask'

In [16]:
def loss(y_pred,y_true, x_pred,x_true):
    losses=tf.keras.losses.MSE(y_pred,y_true)+tf.keras.losses.MSE(x_pred,x_true)
    return losses

def optimizer(x,y):
    with tf.GradientTape() as g:
        model=build(var,dynamic)
        var=model(var)
        y_pred=var[1]
        x_pred=var[3]
        y_true=orig_var[1]
        x_true=orig_var[3]
        losses=loss(y_pred,y_true,x_pred,x_true)
        trainable_variables = weights.values() + biases.values()

In [ ]:
class MyLayer(Layer):
    def __init__(self, axis, **kwargs):
        self.axis = axis
        self.result = None
        super(MyLayer, self).__init__(**kwargs)

    # first use build function to define parameters, Creates the layer weights.
    # input_shape will automatic collect input shapes to build layer
    def build(self, input_shape):
        print(input_shape)
        super(MyLayer, self).build(input_shape)

    # This is where the layer's logic lives. In this example, I just concat two tensors.
    def call(self, inputs, **kwargs):
        a = inputs[0]
        b = inputs[1]
        self.result = K.concatenate([a, b], axis=self.axis)
        return self.result

    # return output shape
    def compute_output_shape(self, input_shape):
        return K.int_shape(self.result)
var_1 = Input(shape=(1,))
var_2 = Input(shape=(2,))

var_test = MyLayer(axis=1)([var_1, var_2])

model = Model(inputs=[var_1, var_2], outputs=[var_test])
model.compile(optimizer='adam', loss='mse')

print(model.summary())